In [5]:
import pandas as pd
import numpy as np
import os, sys, regex

In [6]:
dala = os.getenv('HOME')

In [7]:
base_dir = f'{dala}/Documents/RunSheets/AllRunSheets'

In [8]:
cols1 = ['Sample_Name', 'Reporter', 'CT', 'Ct_Mean']

data = pd.DataFrame(columns=cols1)
data_list = []
for file in os.listdir(base_dir):
    if file.endswith('.xls'):
        data_list.append(file)
    pass
sheets = sorted(data_list)

In [9]:
len(np.unique(np.array(sheets)))

476

In [10]:
cols2 = ['Sample Name', 'Reporter', 'CT', 'Ct Mean', 'Cт', 'Cт Mean']

counts = 0

for sheet in sheets:
    df = pd.read_excel(f'{base_dir}/{sheet}')
    count2 = 0
#     print(sheet)
    for row in df.itertuples():
        count2 += 1
        if row[1] == 'Well':
            df = pd.read_excel(f'{base_dir}/{sheet}', 'Results', header=count2)
            cols3 = list(set(df.columns) & set(cols2))
#             print(cols3)
#             print(list(df.columns))
            df2 = df[sorted(cols3)]
            df2.columns = sorted(cols1)
            data = data.append(df2, ignore_index=True)
            counts += 1
            break
        else: pass

In [11]:
print(f'number of sheets in folder: {len(sheets)}')
print(f'data rows: {data.shape[0]}')
print(f'sheets processed: {counts}')

number of sheets in folder: 476
data rows: 96338
sheets processed: 476


In [12]:
data.head(15)

,Sample_Name,Reporter,CT,Ct_Mean
0,COVC 13541,FAM,Undetermined,NaN
1,COVC 13541,VIC,Undetermined,NaN
2,COVC 13541,CY5,21.914249,21.914249
3,COVC 13542,FAM,Undetermined,NaN
4,COVC 13542,VIC,Undetermined,NaN
5,COVC 13542,CY5,24.626238,24.626238
6,COVC 13543,FAM,Undetermined,NaN
7,COVC 13543,VIC,Undetermined,NaN
8,COVC 13543,CY5,21.624031,21.624031
9,COVC 13544,FAM,Undetermined,NaN


In [13]:
data2 = data[data['CT'] != 'Undetermined']

In [14]:
data3 = data2[data2['Reporter'].isin(['FAM', 'VIC', 'ABY']) == True]

In [15]:
data1 = data3[data3['Sample_Name'].isin(['PC', 'NC']) != True]

In [16]:
data4 = data1[data1.duplicated('Sample_Name', False) == True]

In [17]:
data4.head(10)

,Sample_Name,Reporter,CT,Ct_Mean
126,COVC 13583,FAM,37.328472,37.328472
127,COVC 13583,VIC,42.683975,42.683975
135,COVC 13586,FAM,35.068436,35.068436
136,COVC 13586,VIC,39.385445,39.385445
186,COVC 13603,FAM,27.376181,27.376181
187,COVC 13603,VIC,31.060112,31.060112
219,COVC 13614,FAM,36.698231,36.698231
231,COVC 13618,FAM,32.132778,32.132778
232,COVC 13618,VIC,36.225323,36.225323
261,COVC 13628,FAM,33.411869,33.411869


In [18]:
def cov_rename(x):
    try:
        if 'COV' not in x and '-R' in x:
            x = x.replace('-R', '')
        if 'COV' in x and 'COVC' not in x:
            x = x.replace('COV', 'COVC')
        int(x)
        x = str(x).replace(str(x), f'COVC{str(x)}')
    except (ValueError, AttributeError, TypeError): x = str(x).replace(' ', '')
    return x

In [19]:
regex

<module 'regex' from '/home/douso/.local/lib/python3.8/site-packages/regex/__init__.py'>

In [20]:
def cov_names(x):
    x = str(x).upper().strip()
    if len(x) == 8:
        x = x.replace('COVC', 'COVC0')
    elif len(x) == 7:
        x = x.replace('COVC', 'COVC00')
    elif len(x) == 6:
        x = x.replace('COVC', 'COVC000')
    elif len(x) == 5:
        x = x.replace('COVC', 'COVC0000')
    return x

In [21]:
data4['Reporter'].unique()

array(['FAM', 'VIC', 'ABY'], dtype=object)

In [22]:
data5 = data4.assign(Sample_Name=data4['Sample_Name'].apply(lambda x: cov_names(cov_rename(x))))

In [23]:
data5.head()

,Sample_Name,Reporter,CT,Ct_Mean
126,COVC13583,FAM,37.328472,37.328472
127,COVC13583,VIC,42.683975,42.683975
135,COVC13586,FAM,35.068436,35.068436
136,COVC13586,VIC,39.385445,39.385445
186,COVC13603,FAM,27.376181,27.376181


In [24]:
data6 = data5.sort_values(['Sample_Name', 'Reporter']).groupby('Sample_Name').mean()

In [25]:
data6#.head()

,Ct_Mean
Sample_Name,
A1,33.064911
BLANKEXTRACTION,32.781662
C3,33.114729
COVC00137,37.646282
COVC00154,35.240269
...,...
SEWAGE2_KARBANGI,37.252859
STD1_2X10^4,25.153202
STD3_2X10^2,30.812906


In [36]:
data6.reset_index().to_excel(f'{base_dir}/cov-positiveCts.xlsx', index=False)

In [27]:
#data8.to_excel('cov-positivesCts.xlsx', index=False)

In [30]:
# biorepo = pd.read_excel('/home/douso/Documents/RunSheets/BioRepository/COVID -19 SAMPLE ARCHIVE.xlsx')

In [31]:
# biorepo.head()

,Animal ID,Sample Type,Date of sample collection,Date Frozen,Volume aliquoted,Freezer No.,ALIQUOT A Sample ID,ALIQUOT A Storage box,Position in box,ALIQUOT B Sample ID,ALIQUOT B Storage box,Position in box.1,Aliquoted by,Owner,Experiment,Project,Compiled by,Project code,Unnamed: 18
0,Human,Respiratory in VTM,NaT,NaT,Approx 1.2ml,BSL3 Freezer,COVA00001,SARS0001,1A,COVB0001,SARS0002,1A,NaN,Samuel/Edward/MOH,SARS-CoV-2 rt RT-PCR,COVID Diagnosis,stephen,BMZ002501 - COVID,NaN
1,Human,Respiratory in VTM,NaT,NaT,Approx 1.2ml,BSL3 Freezer,COVA00002,SARS0001,2A,COVB0002,SARS0002,2A,NaN,Samuel/Edward/MOH,SARS-CoV-2 rt RT-PCR,COVID Diagnosis,stephen,BMZ002501 - COVID,NaN
2,Human,Respiratory in VTM,NaT,NaT,Approx 1.2ml,BSL3 Freezer,COVA00003,SARS0001,3A,COVB0003,SARS0002,3A,NaN,Samuel/Edward/MOH,SARS-CoV-2 rt RT-PCR,COVID Diagnosis,stephen,BMZ002501 - COVID,NaN
3,Human,Respiratory in VTM,NaT,NaT,Approx 1.2ml,BSL3 Freezer,COVA00004,SARS0001,4A,COVB0004,SARS0002,4A,NaN,Samuel/Edward/MOH,SARS-CoV-2 rt RT-PCR,COVID Diagnosis,stephen,BMZ002501 - COVID,NaN
4,Human,Respiratory in VTM,NaT,NaT,Approx 1.2ml,BSL3 Freezer,COVA00005,SARS0001,5A,COVB0005,SARS0002,5A,NaN,Samuel/Edward/MOH,SARS-CoV-2 rt RT-PCR,COVID Diagnosis,stephen,BMZ002501 - COVID,NaN


In [32]:
cols = ['ALIQUOT A Sample ID', 'ALIQUOT A Storage box', 'Position in box',
       'ALIQUOT B Sample ID', 'ALIQUOT B Storage box', 'Position in box.1']

In [33]:
biorepo_loc = biorepo[cols]

In [34]:
biorepo_loc.head()

,ALIQUOT A Sample ID,ALIQUOT A Storage box,Position in box,ALIQUOT B Sample ID,ALIQUOT B Storage box,Position in box.1
0,COVA00001,SARS0001,1A,COVB0001,SARS0002,1A
1,COVA00002,SARS0001,2A,COVB0002,SARS0002,2A
2,COVA00003,SARS0001,3A,COVB0003,SARS0002,3A
3,COVA00004,SARS0001,4A,COVB0004,SARS0002,4A
4,COVA00005,SARS0001,5A,COVB0005,SARS0002,5A


In [37]:
cts = pd.read_excel(f'{base_dir}/cov-positiveCts-dedup.xlsx', '30AndBelowCts')

In [38]:
df_rslt = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx')

In [39]:
df_rslt['RESULT'].unique()

array(['Negative', 'Positive', nan, 'neg', 'pos', 'negative', 'positive',
       'Neg', 'Pos', 'Inconclusive', 'MS', 'Positve', 'Positive '],
      dtype=object)

In [40]:
df_rslt_pos = df_rslt['']

KeyError: ''

In [ ]:
cts.head()

In [ ]:
def cov_names(x):
    x = x.upper()
    if len(x) == 9:
        x = x.replace('COVC', 'COVA')
    elif len(x) == 8:
        x = x.replace('COVC', 'COVA0')
    elif len(x) == 7:
        x = x.replace('COVC', 'COVA00')
    elif len(x) == 6:
        x = x.replace('COVC', 'COVA000')
    elif len(x) == 5:
        x = x.replace('COVC', 'COVA0000')
    return x

In [ ]:
def cov_namesA(x):
    x = str(x)
    if len(x) == 8:
        x = x.replace('COVA', 'COVA0')
    elif len(x) == 7:
        x = x.replace('COVA', 'COVA00')
    elif len(x) == 6:
        x = x.replace('COVA', 'COVA000')
    elif len(x) == 5:
        x = x.replace('COVA', 'COVA0000')
    elif len(x) == 4:
        x = x.replace('COVA', 'COVA00000')
    elif len(x) == 3:
        x = x.replace('COVA', 'COVA000000')
    elif len(x) == 2:
        x = x.replace('COVA', 'COVA0000000')
    return x

In [ ]:
cts_cln = cts.assign(Sample_NameB=cts['Sample_Name'].apply(lambda x: cov_names(x)))

In [ ]:
cts_cln

In [ ]:
biorepo_loc_cln = biorepo_loc.assign(ALIQUOT_A_Sample_ID=biorepo_loc['ALIQUOT A Sample ID'].apply(lambda x: cov_namesA(x)))

In [ ]:
biorepo_loc_cln.head()

In [ ]:
# biorepo_loc_cln.to_excel('/home/douso/Documents/RunSheets/BioRepository/COVID-19-Sample-Archive-loc-cln.xlsx')

In [ ]:
cts_cln.head()

In [ ]:
cts_biorepo = cts_cln.set_index('Sample_NameB').merge(biorepo_loc_cln.set_index('ALIQUOT_A_Sample_ID'), how='inner', left_index=True, right_index=True)

In [ ]:
cts_biorepo.head()

In [ ]:
seq_all = pd.read_excel('/home/douso/Documents/RunSheets/all-(un)sequenced-covs.xlsx')

In [ ]:
seq_all.head()

In [ ]:
forseq = pd.read_excel('/home/douso/Documents/RunSheets/all-selected-for-sequencing-covs.xlsx')

In [ ]:
forseq

In [ ]:
df_mar_may = pd.read_excel('/home/douso/Documents/RunSheets/covs_mar-may.xlsx')

In [ ]:
df_mar_may.head()

In [ ]:
df_mar_may[df_mar_may['covc'].isin(list(forseq['covc_code'])) == False]

In [ ]:
sequenced = list(seq_all['covcs'].apply(lambda x: cov_names(x)))

In [ ]:
cts_biorepo_srt = (cts_biorepo.reset_index().sort_values('index'))

In [ ]:
cts_biorepo_srt

In [ ]:
final = cts_biorepo_srt[cts_biorepo_srt['index'].isin(sequenced) == False].sort_values(['index', 'Ct_Mean']).drop_duplicates('index', keep='last')

In [ ]:
final#.head()

In [ ]:
final.to_excel('/home/douso/Documents/RunSheets/BioRepository/cov-positiveCtsLocs.xlsx', index=False)

In [56]:
df_dts_cln_pos = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-pos.xlsx')

In [58]:
l = list(df_dts_cln_pos['S_NUM'])

In [61]:
final_flt = final[final['Sample_Name'].isin(l) == True]

In [62]:
final_flt.to_excel('/home/douso/Documents/RunSheets/BioRepository/cov-positiveCtsLocs-flt.xlsx', index=False)